# MISSING VALUE TREATMENT

In [1]:

fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti'

import sys
print(sys.path)
print()
#sys.path.append(f'{fileloc}')
print()
#print(sys.path)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti', '/anaconda/envs/azureml_py38/lib/python38.zip', '/anaconda/envs/azureml_py38/lib/python3.8', '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload', '', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/_vendor']




In [2]:

class Data(object):
    pass

df = Data
summ = Data


In [3]:
import helpdesk as hd

In [4]:

import pandas as pd, numpy as np
from datetime import datetime


In [5]:

DEBUG = True
seed = 1919


In [6]:

pd.set_option('display.max_rows', 50);
pd.set_option('display.max_columns', 50);
pd.set_option('display.width', 200);


In [7]:

raw_dataset_name = 'raw_dataset_2022-10-18.csv'

df_raw_data = pd.read_csv(f'{fileloc}/{raw_dataset_name}')

print(df_raw_data.shape)
print()
print(df_raw_data.columns.values.tolist())


(577317, 14)

['@id_DOG', 'DogName', 'y_runtime', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'Track', 'Event_Dt', 'x_dist', 'df_avg_speed_prev', 'x_races_tot', 'position', 'ft_sp']


In [8]:

df_raw_data.isna().sum()


@id_DOG                 0
DogName                 0
y_runtime            5856
TrainerName             0
@id_RACE                0
RaceNum                 0
RaceName               15
Track                   0
Event_Dt                0
x_dist                  0
df_avg_speed_prev     348
x_races_tot             0
position                0
ft_sp                4482
dtype: int64

In [9]:

#df_raw_data.groupby('position').count().rsub(df.groupby('position').size(), axis=0)

df_raw_data.groupby('position', dropna = False).agg({'df_avg_speed_prev': lambda x: x.isna().sum()})


,df_avg_speed_prev
position,
1,44.0
2,43.0
3,50.0
4,42.0
5,52.0
6,40.0
7,35.0
8,30.0
B,0.0


In [10]:

'''
Predicted Variable 
    0.99% y_runtime missing =  2445 records
'''

print(df_raw_data.shape)
df_raw_data = df_raw_data[~df_raw_data.y_runtime.isna()]
print(df_raw_data.shape)


(577317, 14)
(571461, 14)


In [11]:

df_raw_data.isna().sum()


@id_DOG                 0
DogName                 0
y_runtime               0
TrainerName             0
@id_RACE                0
RaceNum                 0
RaceName               15
Track                   0
Event_Dt                0
x_dist                  0
df_avg_speed_prev     336
x_races_tot             0
position                0
ft_sp                2934
dtype: int64

In [12]:

'''
df_avg_speed_prev is missing = first race
    create another variable for first race
    impute missing value as the average of runtime for [trainer, x_rac_tot = 1]
'''

#print( df_raw_data.x_races_tot.value_counts() )

df_raw_data['x_no_prev_races'] = df_raw_data.apply( lambda x : 1 if pd.isna(x.df_avg_speed_prev) else 0, axis = 1 )

print( df_raw_data['x_no_prev_races'].value_counts() )



0    571125
1       336
Name: x_no_prev_races, dtype: int64


In [13]:

'''
Imputing the average speed as the "average speed of ALL greyhounds on their first race at the track"
'''

#replace_group = ['TrainerName','x_races_tot']
#replace_group = ['Track','x_races_tot']
replace_group = ['Track']

df_mvt_avg_speed = df_raw_data[ (df_raw_data.x_races_tot == 0) & ~(df_raw_data.df_avg_speed_prev.isna()) ].groupby( replace_group ).agg( {'df_avg_speed_prev':'mean'} ).rename( columns = { 'df_avg_speed_prev' : 'avg_speed_impute' } )

print(df_raw_data.shape)

df_raw_data_impute = pd.merge( df_raw_data, df_mvt_avg_speed, on = replace_group, how = 'left')

df_raw_data_impute['x_avg_speed_prev'] = df_raw_data_impute.apply(lambda x : x.avg_speed_impute if pd.isna(x.df_avg_speed_prev) else x.df_avg_speed_prev, axis = 1)

print(df_raw_data_impute.shape)
print(df_raw_data_impute.columns.values.tolist())


(571461, 15)
(571461, 17)
['@id_DOG', 'DogName', 'y_runtime', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'Track', 'Event_Dt', 'x_dist', 'df_avg_speed_prev', 'x_races_tot', 'position', 'ft_sp', 'x_no_prev_races', 'avg_speed_impute', 'x_avg_speed_prev']


In [14]:

df_raw_data_impute.isna().sum()


@id_DOG                 0
DogName                 0
y_runtime               0
TrainerName             0
@id_RACE                0
RaceNum                 0
RaceName               15
Track                   0
Event_Dt                0
x_dist                  0
df_avg_speed_prev     336
x_races_tot             0
position                0
ft_sp                2934
x_no_prev_races         0
avg_speed_impute        1
x_avg_speed_prev        0
dtype: int64

In [15]:

df_raw_data_impute[df_raw_data_impute.df_avg_speed_prev.isna()]


,@id_DOG,DogName,y_runtime,TrainerName,@id_RACE,RaceNum,RaceName,Track,Event_Dt,x_dist,df_avg_speed_prev,x_races_tot,position,ft_sp,x_no_prev_races,avg_speed_impute,x_avg_speed_prev
0,108390981,DOLLY DOES IT,26.43,M Elphinstone,626271182,10,GREAT NORTHERN,Devonport,2021-01-05,452,NaN,0,4,11.0,1,17008.287271,17008.287271
243,118021641,SKY FLIX,17.16,G Murphy,695991216,3,TAB VENUE MODE VETERANS (225+ RANK),Healesville,2021-07-27,300,NaN,0,4,3.8,1,17093.175768,17093.175768
1602,142710352,THOMELI BOY,22.41,K Howard,632086656,12,PREMIER SPEEDWAY WARRNAMBOOL,Warrnambool,2021-01-14,390,NaN,1,3,28.3,1,17074.576808,17074.576808
1948,151480356,MEPUNGA ZEUS,23.57,J Hosking,634972725,4,SENNACHIE @ STUD - STEVE WHITE,Albion Park,2021-02-01,395,NaN,0,6,7.5,1,16942.076433,16942.076433
5018,198401524,YUKEN TOUCAN,22.78,R Dannatt,650237526,6,BROKEN HILL GAP,Broken Hill,2021-03-19,375,NaN,0,5,13.0,1,16990.912320,16990.912320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569715,685382530,STYLISH RICO,19.40,B Kline,822917301,2,TAB - VENUE MODE,Albion Park,2022-09-11,331,NaN,1,1,8.0,1,16942.076433,16942.076433
569958,694776834,PESSIMISM,22.26,J Shaw,816959162,2,BYERS ELECTRICAL,Shepparton,2022-08-18,385,NaN,1,2,4.4,1,17083.437142,17083.437142
571166,1927740095,BUDDY BANQUET,21.27,G Apps,650115358,3,TRIMECH ELECTRICAL,Young,2021-03-20,347,NaN,1,7,6.0,1,16939.218420,16939.218420
571299,2071430017,MOUNT OXLEY,20.66,C Micallef,629704160,3,LADBROKES MASTERS,Goulburn,2021-01-15,350,NaN,1,4,20.4,1,16997.901630,16997.901630


In [16]:

#keep_cols_for_export = ['@id_DOG', 'DogName', '@id_RACE','RaceName','Track','RaceGrade','Distance','TrainerName','RunTime','Place','x_races_tot']

# 'TrainerName', 'Track' have been dummified
cols_for_drop = [ 'df_avg_speed_prev' , 'avg_speed_impute' ]

todaydt = datetime.now().date()
exp_fname = f'modeling_dataset_{todaydt}.csv'

exp_mod_ds = df_raw_data_impute.drop(columns = cols_for_drop).rename(columns = { 'RunTime' : 'y_runtime' })


In [17]:

print(exp_mod_ds.columns.values.tolist())


['@id_DOG', 'DogName', 'y_runtime', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'Track', 'Event_Dt', 'x_dist', 'x_races_tot', 'position', 'ft_sp', 'x_no_prev_races', 'x_avg_speed_prev']


In [18]:

exp_mod_ds.describe()


,@id_DOG,y_runtime,@id_RACE,RaceNum,x_dist,x_races_tot,ft_sp,x_no_prev_races,x_avg_speed_prev
count,5.714610e+05,571461.000000,5.714610e+05,571461.000000,571461.000000,571461.000000,568527.000000,571461.000000,571461.000000
mean,4.529896e+08,24.443111,7.291770e+08,6.131724,417.516905,18.628736,16.906498,0.000588,17092.930864
std,1.122953e+08,4.980097,5.831171e+07,3.286222,83.848594,16.088104,20.979025,0.024241,331.998650
min,1.083910e+08,10.000000,6.238619e+08,1.000000,259.000000,0.000000,0.260000,0.000000,11387.473779
25%,3.836047e+08,20.580000,6.798781e+08,3.000000,350.000000,6.000000,4.200000,0.000000,16911.877249
50%,4.517682e+08,23.540000,7.290297e+08,6.000000,400.000000,14.000000,8.700000,0.000000,17115.511765
75%,5.283827e+08,27.260000,7.792435e+08,9.000000,461.000000,27.000000,20.500000,0.000000,17294.050131
max,2.145360e+09,396.120000,8.292517e+08,18.000000,732.000000,126.000000,999.990000,1.000000,40200.000000


In [19]:

exp_mod_ds.to_csv(f'{fileloc}/{exp_fname}', index = False)
